# Wstęp
Poza tym notebookiem, który zawiera wszystkie operacje na danych jakie wykonałem w tej analizie, załączam również poglądowo notebook z algorytmem budującym analizowany zbiór danych (<i>[praca_magisterska] data_preparation.ipynb</i>). Kompilacja go wymaga podania klucza do API od Riot (producent LoL'a) - na Pana prośbę mogę go wygenerować i podesłać, jeśli chciałby Pan przejrzeć jak wyciągnąłem dane. Klucz autentykacyjny traci ważność po 24 godzinach, z tego względu obecnie kod się nie skompiluje.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_columns", 100)

In [ ]:
dane = pd.read_csv('data_raw.csv')

In [ ]:
dane.shape

# Czyszczenie danych

Pierwotny zbiór danych, który wygenerowałem zawierał ponad 96 tysięcy obserwacji i 43 zmienne. Poniżej widać 5 poglądowych rekordów. Każda obserwacja to statystyki 1 gracza z 1 meczu. 

In [ ]:
dane.head()

W pierwszej kolejności chciałem ustalić, która z potencjalnych zmiennych, <b>role, individualPosition lub teamPosition</b>, będzie lepszą zmienną do jednoznacznego określenia roli jaką dany gracz odgrywał w meczu. Im bardziej zbalansowany będzie podział między rolami, tym lepsza jest to zmienna.

In [ ]:
dane[['role','gameId']].groupby(['role']).count()

In [ ]:
dane[['individualPosition','gameId']].groupby(['individualPosition']).count()

In [ ]:
dane[['teamPosition','gameId']].groupby(['teamPosition']).count()

In [ ]:
pd.crosstab([dane['role'],dane['teamPosition']],dane['individualPosition'])

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
ari_dataset = dane[['role','individualPosition','teamPosition']].dropna(0)
ari_dataset.shape

In [ ]:
role_ip = adjusted_mutual_info_score(ari_dataset['role'],ari_dataset['individualPosition'])
role_tp = adjusted_mutual_info_score(ari_dataset['role'],ari_dataset['teamPosition'])
tp_ip = adjusted_mutual_info_score(ari_dataset['teamPosition'],ari_dataset['individualPosition'])
print('Adjusted Mutual Information:')
print('role_ip: '+str(role_ip))
print('role_tp: '+str(role_tp))
print('tp_ip: '+str(tp_ip))

In [ ]:
role_ip = adjusted_rand_score(ari_dataset['role'],ari_dataset['individualPosition'])
role_tp = adjusted_rand_score(ari_dataset['role'],ari_dataset['teamPosition'])
tp_ip = adjusted_rand_score(ari_dataset['teamPosition'],ari_dataset['individualPosition'])
print('Adjusted Rand Index:')
print('role_ip: '+str(role_ip))
print('role_tp: '+str(role_tp))
print('tp_ip: '+str(tp_ip))

Po obliczeniu ARI (Adjusted Rand Index) oraz AMI (Adjusted Mutual Information) dla 3 kombinacji zmiennych widać, że najwyższe wartości obserwujemy dla zmiennej teamPosition - oznacza to, że jest ona najbardziej spójna z obiema pozostałymi zmiennymi, czyli jest syntezą informacji ze wszystkich 3. Dodatkowo jest bardzo zbalansowana - dlatego wybieram ją jako ground truth do późniejszej klasteryzacji.

Na etapie tworzenia zbioru zorientowałem się, że mecze rozgrywane w różnych okresach funkcjonowania gry (a co za tym idzie różnymi wersjami API) były opisywany w inny sposób pod względem liczenia czasu rozgrywki. Zamiast czasu rozgrywki wyrażonego w sekundach posługiwano się milisekundami. Oznacza to, że dla sporej części obserwacji mam źle obliczone wartości statystyk w odniesieniu do czasu rozgrywki np. killsPerMinute.

In [ ]:
dane['gameDurationMin'].hist(bins=100)

In [ ]:
dane['gameDurationMin'].hist(bins=20)

In [ ]:
poprawne_czasy = dane[dane['gameDurationMin']>=5].reset_index(drop=True)

In [ ]:
dane['gameDurationMin'][dane['gameDurationMin']<5].hist(bins=20)

Aby naprawić statystyki rozgrywki trzeba dla tych meczy, które mają krótki czas przemnożyć gameDurationMin razy 1000, a statystyki per Minute podzielić na 1000.

In [ ]:
do_poprawy = dane[dane['gameDurationMin']<3].reset_index(drop=True)
do_poprawy['longestTimeSpentLiving'].hist(bins=100)

W zbiorze danych do poprawy dla niektorych rekordow (ponizej 1) trzeba zmienną longestTimeSpentLiving przemnożyć przez 1000.

In [ ]:
do_poprawy['gameDurationMin'] = do_poprawy['gameDurationMin']*1000
do_poprawy['killsPerMinute'] = do_poprawy['killsPerMinute']/1000
do_poprawy['deathsPerMinute'] = do_poprawy['deathsPerMinute']/1000
do_poprawy['assistsPerMinute'] = do_poprawy['assistsPerMinute']/1000
do_poprawy['damageDealtPerMinute'] = do_poprawy['damageDealtPerMinute']/1000
do_poprawy['damageDealtToChampionsPerMinute'] = do_poprawy['damageDealtToChampionsPerMinute']/1000
do_poprawy['damageTakenPerMinute'] = do_poprawy['damageTakenPerMinute']/1000
do_poprawy['damageSelfMitigatedPerMinute'] = do_poprawy['damageSelfMitigatedPerMinute']/1000
do_poprawy['magicDamageDealtPerMinute'] = do_poprawy['magicDamageDealtPerMinute']/1000
do_poprawy['physicalDamageDealtPerMinute'] = do_poprawy['physicalDamageDealtPerMinute']/1000
do_poprawy['goldEarnedPerMinute'] = do_poprawy['goldEarnedPerMinute']/1000
do_poprawy['goldSpentPerMinute'] = do_poprawy['goldSpentPerMinute']/1000
do_poprawy['minionsKilledPerMinute'] = do_poprawy['minionsKilledPerMinute']/1000
do_poprawy['wardsPlacedPerMinute'] = do_poprawy['wardsPlacedPerMinute']/1000
do_poprawy['wardsKilledPerMinute'] = do_poprawy['wardsKilledPerMinute']/1000
do_poprawy['inhibitorKillsPerMinute'] = do_poprawy['inhibitorKillsPerMinute']/1000
do_poprawy['timeCrowdControlDealtPerMinute'] = do_poprawy['timeCrowdControlDealtPerMinute']/1000
do_poprawy['experiencePerMinute'] = do_poprawy['experiencePerMinute']/1000
do_poprawy['damageDealtToTurretsPerMinute'] = do_poprawy['damageDealtToTurretsPerMinute']/1000
do_poprawy['ultimateCastsPerMinute'] = do_poprawy['ultimateCastsPerMinute']/1000
do_poprawy['totalHealPerMinute'] = do_poprawy['totalHealPerMinute']/1000
do_poprawy['turretKillsPerMinute'] = do_poprawy['turretKillsPerMinute']/1000

In [ ]:
poprawione_do_concat = do_poprawy[do_poprawy['longestTimeSpentLiving']>=0.1].reset_index(drop=True)

In [ ]:
poprawione_do_concat['longestTimeSpentLiving'].hist(bins=100)

In [ ]:
reszta = do_poprawy[do_poprawy['longestTimeSpentLiving']<0.1].reset_index(drop=True)

In [ ]:
reszta['longestTimeSpentLiving'].hist(bins=100)

In [ ]:
reszta['longestTimeSpentLiving'] = reszta['longestTimeSpentLiving']*1000

In [ ]:
reszta['longestTimeSpentLiving'].hist(bins=100)

In [ ]:
reszta_do_odratowania = reszta[reszta['longestTimeSpentLiving']>=10].reset_index(drop=True)

In [ ]:
dane = pd.concat([poprawne_czasy,poprawione_do_concat,reszta_do_odratowania]).drop_duplicates().reset_index(drop=True)

In [ ]:
dane['gameDurationMin'].hist(bins=100)

In [ ]:
dane['longestTimeSpentLiving'].hist(bins=100)

Teraz dane o czasie gry i najdluzszym czasie bez życia gracza mają sens. W następnych krokach odfiltrowuję za krótkie mecze (poniżej 15 minut - dopiero wtedy mecz może się w normalnych warunkach zakończyć wygraną którejś ze stron poprzez poddanie tej drugiej.

In [ ]:
dane = dane[dane['gameDurationMin']>=15].reset_index(drop=True)

In [ ]:
dane['damageDealtPerMinute'].hist(bins=100)

In [ ]:
dane[dane['damageDealtPerMinute']==0].reset_index(drop=True)

In [ ]:
dane = dane[dane['damageDealtPerMinute']>0].reset_index(drop=True)

In [ ]:
dane.shape

In [ ]:
dane.describe().transpose()

Usuwam zmienne, które posiadają niezrozumiałe dane: damageRatio - nieskończoność gdy gracz nie otrzymal zadnych obrazen oraz timeCrowdControlDealtPerMinute - mam wątpliwości czy te dane są dobrze policzone.

In [ ]:
dane_filtered_columns = dane.loc[:, ~dane.columns.isin(['damageRatio', 'timeCrowdControlDealtPerMinute'])]

Ostatnim etapem jest usunięcie obserwacji z brakującymi danymi.

In [ ]:
dane_filtered_columns.isnull().values.any()

In [ ]:
dane_filtered_columns.isnull().sum().sum()


In [ ]:
dane_filtered_columns[dane_filtered_columns.isna().any(axis=1)]

In [ ]:
dane_filtered_columns.columns[dane_filtered_columns.isnull().any()]

Kolejnym powodem, żeby usunąć te obserwacje jest fakt, że nie mają one bardzo istotnej zmiennej - teamPosition, którą wybrałem jako identyfikator roli gracza.

In [ ]:
dane_filtered_columns = dane_filtered_columns.dropna()

In [ ]:
dane_filtered_columns.groupby(['teamPosition']).count()

In [ ]:
dane_filtered_columns = dane_filtered_columns.reset_index(drop=True)

In [ ]:
dane_filtered_columns.shape

Ostateczny zbiór, z którym przechodzę do etapu PCA ma 93.5 tysiąca obserwacji i 41 zmiennych.

# Redukcja wielowymiarowości - PCA

W związku z tak dużą liczbą wymiarów i zależnościami między nimi mam hipotezę, że w zbiorze zachodzi współliniowość i konieczna będzie redukcja wielowymiarowości, którą przeprowadzę metodą PCA.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 7,7 
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d
import numpy as np
from heatmap import heatmap, corrplot
sns.set(color_codes=True, font_scale=1.2)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
data = dane_filtered_columns

In [ ]:
plt.figure(figsize=(20, 20))
corrplot(data.corr(), size_scale=300)

Czyli potwierdzona hipoteza - mamy zjawisko współliniowości, przykłady:
<br> <b>percentMagic - percentPhysical</b> silna odwrotna korelacja (oczywiste, bo dopelnieniem jednego jest drugie)
<br> <b>playerRank - experiencePerMinute</b> silna odwrotna korelacja (sam tak stworzylem zmienna, wiec tez bez zaskoczen)
<br> <b>champLevel - gameDurationMin</b> silna korelacja (ciekawe, im wyzszy poziom graczy tym mecze sa bardziej zaciete)

Przed nastepnymi krokami musze rozdzielic dane na zmienne liczbowe, ktore bede wykorzystywac w PCA oraz w klasteryzacji i na zmienne "na potem". Zmienne liczbowe muszę wyskalować przed PCA.

In [ ]:
df_stats = data.loc[:, ~data.columns.isin(['gameId', 'teamId','lane','role','playerRank','individualPosition','teamPosition','win','gameDurationMin','gameEndedInSurrender','championName'])]

In [ ]:
len(df_stats.columns)

In [ ]:
sc = StandardScaler()
sc.fit(df_stats)
data_std = sc.transform(df_stats)
pca = PCA()

In [ ]:
data_pca = pca.fit_transform(data_std)

exp_var_pca = pca.explained_variance_ratio_
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.rcParams["figure.figsize"] = (16,8)
plt.bar(range(0+1,len(exp_var_pca)+1), exp_var_pca, alpha=0.5, align='center', label='Wyjaśniona wariancja')
plt.step(range(0+1,len(cum_sum_eigenvalues)+1), cum_sum_eigenvalues, where='mid',label='Skumulowana wyjaśniona wariancja')
plt.ylabel('Wariancja',fontsize = 20)
plt.xlabel('Indeks głównej składowej',fontsize = 20)
plt.legend(loc='best',fontsize = 20)
plt.tight_layout()
plt.figure(figsize=(40, 40))
plt.show()

In [ ]:
cum_sum_eigenvalues

Dla celów dalszej analizy biorę 5 komponentów, które łącznie wyjaśniają ponad 62% wariancji zbioru.

In [ ]:
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal_component_1', 'principal_component_2', 'principal_component_3','principal_component_4','principal_component_5'])

Wizualizacje komponentów w 2 i 3D.

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,20)
plt.scatter(principalDf['principal_component_1'], principalDf['principal_component_2'],c='blue', s=70, alpha=0.005)
plt.ylabel('Główna składowa nr 1',fontsize=20)
plt.xlabel('Główna składowa nr 2',fontsize=20)
plt.show()

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,20)
plt.scatter(principalDf['principal_component_1'], principalDf['principal_component_3'],c='blue', s=70, alpha=0.005)
plt.ylabel('Główna składowa nr 1',fontsize=20)
plt.xlabel('Główna składowa nr 3',fontsize=20)
plt.show()

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,20)
plt.scatter(principalDf['principal_component_2'], principalDf['principal_component_3'],c='blue', s=70, alpha=0.005)
plt.ylabel('Główna składowa nr 2',fontsize=20)
plt.xlabel('Główna składowa nr 3',fontsize=20)
plt.show()

Oceniajac wizualnie jedynie 1 i 2 komponent widać potencjalne 3 klastry, jednak po dodaniu do tego 3 komponentu wyłonić się może większa ich liczba. Dlatego będę musiał inną metodą ustalić optymalną liczbę klastrów. Poniżej wykres 3D.

In [ ]:
fig = plt.figure(figsize = (20, 20))
ax = plt.axes(projection ="3d")

ax.scatter3D(principalDf['principal_component_1'], principalDf['principal_component_2'], principalDf['principal_component_3'], color = "blue",alpha=0.003,s=80)
plt.title("3D Principal components visualization")
ax.set_xlabel('principal_component_1')
ax.set_ylabel('principal_component_2')
ax.set_zlabel('principal_component_3')
ax.set_xlim3d(np.percentile(principalDf['principal_component_1'],[1,99])[0],np.percentile(principalDf['principal_component_1'],[1,99])[1])
ax.set_ylim3d(np.percentile(principalDf['principal_component_2'],[1,99])[0],np.percentile(principalDf['principal_component_2'],[1,99])[1])
ax.set_zlim3d(np.percentile(principalDf['principal_component_3'],[1,99])[0],np.percentile(principalDf['principal_component_3'],[1,99])[1])
plt.show()

W następnym kroku przeprowadzam dekompozycję PCA 1 i 2, żeby wizualnie ocenić, które zmienne mocno do nich kontrybuują.

In [ ]:
pca = PCA()
x_new = pca.fit_transform(data_std)
labels = list(df_stats.columns)
def myplot(score,coeff,labels=labels):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley,c='blue', s=70, alpha=0.005)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'black',alpha = 1)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'black', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'black', ha = 'center', va = 'center')
    plt.xlim(-0.5,0.5)
    plt.ylim(-0.5,0.5)
    plt.xlabel("Główna składowa nr {}".format(1),fontsize=20)
    plt.ylabel("Główna składowa nr {}".format(2),fontsize=20)
    plt.grid(True)


myplot(x_new[:,0:2],np.transpose(pca.components_[0:2, :]))
plt.show()

In [ ]:
train_features = data_std

model = PCA(n_components=5).fit(train_features)
X_pc = model.transform(train_features)
n_pcs= model.components_.shape[0]
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = list(df_stats.columns)
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
dic = {'Główna składowa nr {}'.format(i+1): most_important_names[i] for i in range(n_pcs)}
df = pd.DataFrame(dic.items(),columns = ['Główne składowe','Najistotniejsze zmienne'])

In [ ]:
df

Z dekompozycji PCA wynika, że najistotniejsze zmienne w zbiorze pod względem objaśniania wariancji to:
* goldEarnedPerMinute - liczba pieniedzy zarobiona przez gracza na minutę, 
* percentPhysical - udział obrażeń fizycznych w całości obrażeń oraz 
* damageTakenPerMinute - liczba obrażeń otrzymanych przez gracza na minutę. Wszystkie te zmienne moim zdaniem są intuicyjnymi wskaźnikami z jakim graczem mamy do czynienia - dobrym/złym - zarabiającym dużo czy mało, grającym postacią korzystającą z many i punktów umiejętności czy zadającą dużo obrażeń fizycznych itp. 

<br>Dodatkowo można zaobserwować, że są grupy zmiennych, które zachowują się podobnie i wynikaja z siebie, z ciekawszych par:
* deathsPerMinute <-> emptyItemSlots: im częściej ktoś umiera w grze, tym ma mniej pieniędzy, tym mniej może kupić, tym więcej ma pustych slotów na przedmioty w grze albo
* wardsPlacedPerMinute <-> percentMagic: postacie, ktore maja w LoLu zadanie pilnowania "wizji mapy" odpowiadaja za umieszczanie totemow (przedmiotow, ktore zapewniaja widocznosc obszaru, ktory normalnie jest ukryty) na mapie zwykle sa postaciami poslugujacymi sie magia, stad wraz z rosnieciem udzialu magicznych obrazen, rosnie liczba stawianych totemow
* experiencePerMinute <-> deathsPerMinute: duza liczba smierci nie sprzyja mozliwosci zdobywania doswiadczenia w grze :)

# Klasteryzacja

W pierwszej kolejnosci chce poznac optymalna liczbe klastrow i zweryfikowac hipoteze, ze jest to 5 klastrow - bo tyle mamy predefiniowanych rol w LoLu.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_data = principalDf

In [ ]:
kmeans_data

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(kmeans_data)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    

In [ ]:
fig = plt.figure(figsize = (10, 10))
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('Liczba klastrów k',fontsize = 20)
plt.ylabel('Inercja',fontsize = 20)
plt.xticks(ks)
plt.show()

In [ ]:
for i in range(1,len(inertias)):
    print(inertias[i-1]-inertias[i])

Z metody "lokciowej" moim zdaniem nie wylania sie w oczywisty sposob optymalna liczba klastrow - potencjalnie mogloby to byc 4-6. Zeby byc dokladniejszym posluze sie metoda silhouette.

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid

In [ ]:
np.random.seed(123)

In [ ]:
parameters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

parameter_grid = ParameterGrid({'n_clusters': parameters})
best_score = -1
kmeans_model = KMeans()
silhouette_scores = []

for p in parameter_grid:
    kmeans_model.set_params(**p)    
    kmeans_model.fit(kmeans_data)         
    ss = silhouette_score(kmeans_data, kmeans_model.labels_, sample_size = 1000, random_state=112)   
    silhouette_scores += [ss]      
    print('Parameter:', p, 'Silhouette score:', ss)
    if ss > best_score:
        best_score = ss
        best_grid = p

In [ ]:
fig = plt.figure(figsize = (20, 10))
plt.bar(range(len(silhouette_scores)), list(silhouette_scores), align='center', color='black', width=0.5)
plt.xticks(range(len(silhouette_scores)), list(parameters))
# plt.title('Silhouette Score', fontweight='bold')
plt.xlabel('Liczba klastrów k',fontsize=20)
plt.ylabel('Współczynnik Silhouette',fontsize=20)
plt.show()

Wielokrotnie testowalem rozny root, ale zawsze wychodzi, ze najlepsza liczba klastrow to 4. Zatem w docelowej analizie posluze sie ta liczba jako optymalna liczba. Teraz sprawdze jak klasteryzacja z n=5 bedzie miala sie do rol definiowanych przez gre (zmienna teamPosition).

In [ ]:
np.random.seed(123)

In [ ]:
five_clusters = KMeans(n_clusters=5,random_state = 101)
five_clusters.fit(kmeans_data)
predict=five_clusters.predict(kmeans_data)
centroids = five_clusters.cluster_centers_
kmeans_data['label'] = pd.Series(predict, index=kmeans_data.index)

In [ ]:
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots()

colors = {0:'red', 1:'magenta', 2:'blue', 3:'yellow', 4:'cyan'}

ax.scatter(kmeans_data['principal_component_1'], kmeans_data['principal_component_2'], c=kmeans_data['label'].map(colors),alpha = 0.05)
scatter = ax.scatter(kmeans_data['principal_component_1'], kmeans_data['principal_component_2'], c=kmeans_data['label'].map(colors),alpha = 0.05)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black',marker='X')
ax.set_xlabel('Główna składowa 1',fontsize = 20)
ax.set_ylabel('Główna składowa 2',fontsize = 20)
markers = [plt.Line2D([0,0],[0,0],color=color, linewidth=20, marker='o', linestyle='') for color in colors.values()]
plt.legend(markers, colors.keys(), numpoints=1,fontsize = 15,title = 'Segmenty', loc='upper right')
plt.show()

In [ ]:
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots()

colors = {0:'red', 1:'magenta', 2:'blue', 3:'yellow', 4:'cyan'}

ax.scatter(kmeans_data['principal_component_1'], kmeans_data['principal_component_3'], c=kmeans_data['label'].map(colors),alpha = 0.05)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black',marker='X')
ax.set_xlabel('Główna składowa numer 1')
ax.set_ylabel('Główna składowa numer 3')
plt.show()

In [ ]:
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots()

colors = {0:'red', 1:'magenta', 2:'blue', 3:'yellow', 4:'cyan'}

ax.scatter(kmeans_data['principal_component_2'], kmeans_data['principal_component_3'], c=kmeans_data['label'].map(colors),alpha = 0.05)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black',marker='X')
ax.set_xlabel('Główna składowa numer 2')
ax.set_ylabel('Główna składowa numer 3')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 20))
ax = plt.axes(projection ="3d")

ax.scatter3D(kmeans_data['principal_component_1'], kmeans_data['principal_component_2'], kmeans_data['principal_component_3'], color = kmeans_data['label'].map(colors),alpha=0.03,s=80)
ax.scatter3D(centroids[:,0] , centroids[:,1] , centroids[:,2] , s = 150, color = 'black',marker='X',alpha = 1)
plt.title("simple 3D scatter plot")
ax.set_xlabel('principal_component_1')
ax.set_ylabel('principal_component_2')
ax.set_zlabel('principal_component_3')
ax.set_xlim3d(np.percentile(kmeans_data['principal_component_1'],[1,99])[0],np.percentile(kmeans_data['principal_component_1'],[1,99])[1])
ax.set_ylim3d(np.percentile(kmeans_data['principal_component_2'],[1,99])[0],np.percentile(kmeans_data['principal_component_2'],[1,99])[1])
ax.set_zlim3d(np.percentile(kmeans_data['principal_component_3'],[1,99])[0],np.percentile(kmeans_data['principal_component_3'],[1,99])[1])
plt.show()

Po podziale na 5 sztucznie narzuconych klastrow chce sprawdzic jak maja sie one do rol z poczatkowego zbioru, czy sie pokrywaja w duzym stopniu. Dodatkowo chce poznac co charakteryzuje reprezentantow poszczegolnych segmentow.

In [ ]:
data_initial = dane_filtered_columns
data_initial['cluster'] = pd.Series(predict, index=data_initial.index)

In [ ]:
ami_5_clusters = adjusted_mutual_info_score(data_initial['teamPosition'],data_initial['cluster'])
ari_5_clusters = adjusted_rand_score(data_initial['teamPosition'],data_initial['cluster'])

In [ ]:
x1

In [ ]:
columns

In [ ]:
x1 = data_initial.groupby('cluster').mean(1).reset_index()
columns = x1.columns

count=1
plt.subplots(figsize=(20, 40))
for i in range(0,len(columns)):
    plt.subplot(9,4,count)
    plt.bar(x1.index, x1.iloc[:, i], color="black")
    plt.ylabel(columns[i])
    plt.title(columns[i])
    count+=1

plt.show()

Charakterystyki klastrow:

  1 przeciętniak, w żadnej statystyce się nie wybija ani na plus ani na minus, jedynie zauważalnie częściej gra postaciami nastawionymi na obrażenia fizyczne niż na magiczne, pasuje to do opisu ról TOP/JUNGLE, ale równie dobrze, mogą być to po prostu BOTTOMy, tylko grające słabiej
 <br> 2 asystent, ma duży udzial w zabojstwach, choc tylko w formie asyst, bo nie zadaje za duzo obrazen, dba o wizje mapy poprzez umieszczanie i niszczenie wrogich totemow, ponadprzecietnie duzo obrazen zadaje postaciom, zabija wyjatkowo malo minionow - typowa rola UTILITY (czyli tzw. support)
 <br> 4 zwycięzcy, duzo zabijaja, malo gina, zadaja i przyjmuja duzo obrazen - sa w srodku akcji, zadaja w zdecydowanej wiekszosci obrazenia fizyczne i dominuja w niszczeniu fortyfikacji przeciwnika - po prostu dobrzy gracze, jeśli musiałbym przyporządkować do tego segmentu jakąś konkretną rolę to byłby to BOTTOM (czyli tzw. AD carry)
 <br> 3 niedoświadczeni, uczący się gracze, często kończą swoje gry szybko, bo przegrywają i się poddają, trochę częściej gra postaciami nastawionymi na obrażenia fizyczne niż magiczne
 <br> 0 magicy, obok klastra nr 3 jest to drugi najlepszy gracz w druzynie, zadaje duzo obrazen, udaje mu sie czesto zabijac przeciwnikow, gra prawie wylacznie magicznymi postaciami i takie obrazenia tez u niego przewazaja - typowe charakterystyki roli MIDDLE (czyli tzw. mid)

In [ ]:
counter = data_initial[['cluster','teamPosition','teamId','championName']]
counter[['cluster','teamPosition']].groupby('cluster').count()

Liczebności klastrów są dosyć nierówne, najwięcej mamy przeciętniaków (co zdecydowanie ma sens, potwierdzałoby to też hipotezę o mixie 2-3 ról). Najmniej jest za to asystentów i graczy naprawdę dobrych - co również ma pewien sens. Gdy są wybierane role, nikt nie chce być supportem (przyjeło się myśleć, że ta rola jest nudna i niepotrzebna), a elitarnych graczy z definicji musi być niewielu :)

In [ ]:
pd.pivot_table(counter[['cluster','teamPosition','teamId']],index='teamPosition',columns='cluster',aggfunc='count',margins=True)

Nałożenie klastrów na role pozwala potwierdzić pierwotne przypuszczenia odnośnie zależności segmentów od ról - tzn.:
- klaster 1 mieszanka ról, z najmniejszym udziałem supportów, sytuacja podobna do klastra 3 - gracze przeciętni zdarzają się na każdej pozycji i jest to całkowicie naturalne
- klaster 2 to stricte supporty
- klaster 4 bez silniejszej przynależności do roli, dobry gracz odnajdzie się w każdej z ról (poza supportem), ale zgodnie z podejrzeniami jest tu najwięcej AD carry (BOTTOM)
- klaster 3 jest bardzo zrównoważony pod względem ról - to po prostu są gracze słabi/początkujący, którzy niezależnie od roli radzą sobie kiepsko i muszą zbierać doświadczenie
- klaster 0 przeważają gracze grający na midzie, ale zdażają się też junglerzy i topy

Wrócmy do optymalnej liczby klastrów - 4.

In [ ]:
np.random.seed(10982)

In [ ]:
four_clusters = KMeans(n_clusters=4,random_state=123)
four_clusters.fit(kmeans_data)
predict=four_clusters.predict(kmeans_data)
centroids = four_clusters.cluster_centers_
kmeans_data['label'] = pd.Series(predict, index=kmeans_data.index)

In [ ]:
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots()

colors = {0:'red', 1:'magenta', 2:'blue', 3:'yellow'}

ax.scatter(kmeans_data['principal_component_1'], kmeans_data['principal_component_2'], c=kmeans_data['label'].map(colors),alpha = 0.05)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'black',marker='X')
ax.set_xlabel('Główna składowa 1',fontsize = 20)
ax.set_ylabel('Główna składowa 2',fontsize = 20)
markers = [plt.Line2D([0,0],[0,0],color=color, linewidth=20, marker='o', linestyle='') for color in colors.values()]
plt.legend(markers, colors.keys(), numpoints=1,fontsize = 15,title = 'Segmenty', loc='upper right')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 20))
ax = plt.axes(projection ="3d")

ax.scatter3D(kmeans_data['principal_component_1'], kmeans_data['principal_component_2'], kmeans_data['principal_component_3'], color = kmeans_data['label'].map(colors),alpha=0.03,s=80)
ax.scatter3D(centroids[:,0] , centroids[:,1] , centroids[:,2] , s = 150, color = 'black',marker='X',alpha = 1)
plt.title("simple 3D scatter plot")
ax.set_xlabel('principal_component_1')
ax.set_ylabel('principal_component_2')
ax.set_zlabel('principal_component_3')
ax.set_xlim3d(np.percentile(kmeans_data['principal_component_1'],[1,99])[0],np.percentile(kmeans_data['principal_component_1'],[1,99])[1])
ax.set_ylim3d(np.percentile(kmeans_data['principal_component_2'],[1,99])[0],np.percentile(kmeans_data['principal_component_2'],[1,99])[1])
ax.set_zlim3d(np.percentile(kmeans_data['principal_component_3'],[1,99])[0],np.percentile(kmeans_data['principal_component_3'],[1,99])[1])
plt.show()

In [ ]:
data_initial = dane_filtered_columns
data_initial['cluster'] = pd.Series(predict, index=data_initial.index)

In [ ]:
ami_4_clusters = adjusted_mutual_info_score(data_initial['teamPosition'],data_initial['cluster'])
ari_4_clusters = adjusted_rand_score(data_initial['teamPosition'],data_initial['cluster'])

In [ ]:
print('5 clusters')
print('AMI: '+str(ami_5_clusters))
print('ARI: '+str(ari_5_clusters))
print('--------------------------')
print('4 clusters')
print('AMI: '+str(ami_4_clusters))
print('ARI: '+str(ari_4_clusters))

In [ ]:
x1 = data_initial.groupby('cluster').mean(1).reset_index()
columns = x1.columns
x1 =x1

In [ ]:
x1 = data_initial.groupby('cluster').mean(1).reset_index()
columns = x1.columns

count=1
plt.subplots(figsize=(20, 40))
for i in range(0,len(columns)):
    plt.subplot(9,4,count)
    plt.bar(x1['cluster'], x1.iloc[:, i], color="black")
    plt.ylabel(columns[i])
    plt.title(columns[i])
    count+=1

plt.show()

In [ ]:
counter = data_initial[['cluster','teamPosition','teamId','championName']]
counter[['cluster','teamPosition']].groupby('cluster').count()

In [ ]:
pd.pivot_table(counter[['cluster','teamPosition','teamId']],index='teamPosition',columns='cluster',aggfunc='count',margins=True)

Charakterystyki klastrów w optymalnym wydaniu:

1 bardzo duza liczebnosc, dobrzy gracze, zadajacy duze obrazenia, prawie wylacznie fizyczne, wysoki wspolczynnik zabojstw do smierci - analogicznie do poprzedniej klasteryzacji - to najprawdopodobniej mieszanka BOTTOM, JUNGLE i TOP
<br>3 magicy
<br>2 stricte supporty
<br>0 przegrywajacy, niedoswiadczeni gracze z kazdej pozycji

Zatem obnizenie klastrow do 4 spowodowalo znikniecie segmentu przecietniakow, klastry rzeczywiscie bardziej sie od siebie roznia pod wzgledem charakterystyk, ale niewiele dodaje to informacji o podziale miedzy role - ARI i AMI pozostaja na zblizonym poziomie i wciaz klastry slabo pokrywaja sie z ground truth w postaci pozycji opisanej zmienna <b>teamPosition.

# klasyfikacja (regresja logistyczna)

W zwiazku z tym, ze klastry nie pokrywaja mi sie w zadowalajacym stopniu z rolami poszczegolnych graczy, zdecydowalem, ze policze "archetypy" poszczegolnych rol i policze "niestandardowosc" rozgrywki w porownaniu z archetypami kazdej z rol, ktora powinien odgrywac gracz. W tym celu skorzystalem z 2 komponentow z PCA, na ich podstawie policzylem mediane dla kazdej z rol - bedzie to moj archetyp. Nastepnie na podstawie odleglosci euklidejskiej obliczylem jak niestandardowo ktos gral. Takie odleglosci bede nastepnie wpuszczam do modelu regresji logistycznej ze zmienna celu WIN - chce sprawdzic czy niestandardowosc rozgrywki pomoze mi przewidziec czy ktos wygral czy przegral.

In [ ]:
data_initial = data_initial[['teamPosition','win']]
coordinates = pd.merge(kmeans_data,data_initial,left_index=True,right_index=True).set_index('teamPosition')
meta_calculation = pd.merge(kmeans_data,data_initial,left_index=True,right_index=True)
meta_calculation = meta_calculation.groupby('teamPosition').agg({'principal_component_1': 'median', 'principal_component_2': 'median'})
meta_calculation.columns = ['principal_component_1_meta','principal_component_2_meta']
coordinates = pd.merge(coordinates,meta_calculation,left_index=True,right_index=True)
coordinates['euclidean_distance'] = np.sqrt((coordinates['principal_component_1']-coordinates['principal_component_1_meta']).pow(2)+(coordinates['principal_component_2']-coordinates['principal_component_2_meta']).pow(2))
coordinates = coordinates.reset_index()
coordinates = coordinates[['win','teamPosition','euclidean_distance']]
coordinates

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix

In [ ]:
data = coordinates

In [ ]:
data.groupby(['teamPosition','win']).count()

In [ ]:
data.groupby(['teamPosition','win']).agg({'euclidean_distance':['mean','median']})

In [ ]:
data.groupby('win').agg({'euclidean_distance':['mean','median']})

Zbior jest wysoce zbalansowany - to dobrze. Analizujac roznice w srednich i medianach miedzy zwycieskimi i przegranymi rozgrywkami mozna przypuszczac, ze niestandardowosc rozgrywki (nietrzymanie sie archetypu) zwieksza szanse zwyciestwa, co postaram sie zweryfikowac przy pomocy modelu regresji logistycznej. Zaskakujace jest jednak to, ze w roli, ktora najlatwiej rozpoznac (support) wartosci miary niestandardowosci sa niskie (gracze graja w zblizony sposob) i niewiele roznia sie srednie miedzy zwycieskimi rozgrywkami i przegranymi (niestandardowosc rozgrywki supportu nie powinna miec wplywu na ostateczny wynik meczu).

In [ ]:
X=data['euclidean_distance']
y=data['win']
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train = X_train.values.reshape(-1,1)
y_train = y_train.values.reshape(-1,1)
X_test = X_test.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

Zmienna jest istotna statystycznie i rzeczywiscie wraz ze wzrostem niestandardowosci szanse na zwyciestwo sie zwiekszaja, ale ogolna moc predykcyjna modelu jest bardzo mizerna. Sprobuje jeszcze zweryfikowac jak by to wygladalo dla kazdej roli oddzielnie.

In [ ]:
positions = ['BOTTOM','JUNGLE','MIDDLE','TOP','UTILITY']

for i in range(0,len(positions)):
    print('___________________________')
    print(positions[i])
    print('___________________________')
    temporary = data[data.teamPosition.eq(positions[i])]
    X=temporary['euclidean_distance']
    y=temporary['win']
    logit_model=sm.Logit(y,X)
    result=logit_model.fit()
    print(result.summary2())
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    X_train = X_train.values.reshape(-1,1)
    y_train = y_train.values.reshape(-1,1)
    X_test = X_test.values.reshape(-1,1)
    y_test = y_test.values.reshape(-1,1)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Dla kazdej z rol poza UTILITY (support) zmienna mowiaca o niestandardowosci rozgrywki jest istotna statystycznie i jej wspolczynnik zawsze jest dodatni - co potwierdza tezę, że niestandardowe granie podwyższa szanse zwycięstwa drużyny gracza. Z zastrzeżeniem, że niestandardowa gra supporta nie daje drużynie zauważalnie większych szans zwycięstwa. Jednakże w żadnym przypadku jedynie informacja o odleglosci euklidesowej od archetypu nie wystarczy, żeby efektywnie prognozować zwycięstwo - accuracy = 0.53 to tak naprawde niewiele lepiej niz losowo.

# Podsumowanie

1. Dane zostaly wyczyszczone i przygotowane do wnioskowania i modelowania
2. Potwierdzila sie hipoteza o wspoliniowosci zmiennych
3. Zastosowano redukcje wielowymiarowosci przy uzyciu metody PCA, ktorej 5 komponentow pozwolilo wyjasnic ponad 62% wariancji
4. Określono najistotniejsze zmienne w zbiorze, które najbardziej różnicują rozgrywki graczy
5. Przeanalizowano zbior pod katem optymalnej liczby klastrow w segmentacji
6. Nie potwierdzila sie hipoteza o 5 klastrach zaszytych w danych
7. Przy zastosowaniu sztywnej wartosci n=5 segmentacja jedynie w przypadku 2 klastrow w oczywisty sposob wskazywala na konkretna role (support i mid)
8. Przeanalizowano charakterystyki segmentow przy n=4, zastosowanie optymalnej liczby klastrow nie wplynelo na lepsze nalozenie sie klastrow na predefiniowane role (brak istotnych zmian w ARI i AMI).
9. Opracowano miare niestandardowosci rozgrywki w danej roli
10. Zbadano zaleznosc miedzy miara niestandardowosci i szansa na zwyciestwo
11. Potwierdzila sie hipoteza, ze im bardziej niestandardowa/zaskakujaca gra, tym wieksze prawdopodobienstwo zwyciestwa, jednak tylko ta informacja to za malo, zeby prognozowac rezultaty meczy

#### Komentarze:

Jako poglebienie moznaby stworzyc miare niestandardowosci gry calej druzyny (jakas suma odleglosci euklidesowych w obrebie druzyny x meczu y). <br>
W analizie zostalo pominietych kilka waznych aspektow, takich jak postac, ktora gral dany gracz (jak ona odpowiadala roli, ktora powinien byl przyjac), jakie przedmioty kupowali gracze i gdzie tak faktycznie spedzali swoja rozgrywke (w meczach poczatkujacych czestym procederem jest zamienianie sie miedzy liniami, lepsi gracze proponuja, ze zamienia sie z kims, kto wpadl na lepszego od siebie rywala). 